# HLM-GWR 模拟实验

In [50]:
library(MASS)
library(lme4)
library(lmerTest)
library(ggplot2)
library(GWmodel)
library(dplyr)
library(ggplot2)
library(ggpmisc)

In [102]:
generate.data <- function(n, each, order = c(0,1,2), random = c(0,1,1)) {
    k.g <- 1
    k.h <- 2
    s <- ifelse(length(each) == 1, each, length(each))
    l <- ifelse(length(each) == 1, n*n*each, sum(each))
    if (length(each) == 1) {
        group <- rep(1:(n*n), each = each)
    } else {
        group <- rep(1:(n*n), each)
    }
    X <- mvrnorm(l, rep(0, k.g + k.h), diag(1, k.g + k.h, k.g + k.h))
    for (i in 1:(n*n)) {
        group.i <- which(group == i)
        X[group.i,1] <- mean(X[group.i,1])
    }
    X <- cbind(1, X)
    positions.data <- matrix(0, l, 2)
    positions.beta <- matrix(0, n*n, 2)
    for (i in 1:n) {
        for (j in 1:n) {
            p1 = (i - 1)*n + j
            positions.beta[p1,] = c(i,j)
            for (k in 1:each[p1]) {
                p2 = sum(each[1:(p1 - 1)]) + k
                positions.data[p2,] = c(i,j)
            }
        }
    }
    beta.g <- matrix(0, n*n, k.g)
    for (i in 1:n) {
        for (j in 1:n) {
            p = (i - 1)*n + j
            order.g = order[2]
            beta.g[p, 1] = runif(1, -10, 10) * i^order.g * j^order.g
        }
    }
    beta.g[, 1] = beta.g[, 1] + random[2] * rnorm(n*n)
    beta.h <- matrix(0, n*n, k.h)
    for (i in 1:n) {
        for (j in 1:n) {
            p = (i - 1)*n + j
            order.h = order[3]
            beta.h[p, 1] = runif(1, -10, 10) * i^order.h * j^order.h
        }
    }
    beta.h[, 1] = beta.h[, 1] + random[3] * rnorm(n*n)
    beta.h[, 2] = random[3] * rnorm(n*n)
    beta.intercept <- numeric(n*n)
    for (i in 1:n) {
        for (j in 1:n) {
            p = (i - 1)*n + j
            order.intercept = order[1]
            beta.intercept[p] = runif(1, -10, 10) * i^order.intercept * j^order.intercept
        }
    }
    beta.intercept = beta.intercept + random[1] * rnorm(n*n)
    beta <- cbind(beta.intercept, beta.g, beta.h)
    y <- numeric(l)
    for (i in 1:(n*n)) {
        beta.i <- beta[i,]
        group.i <- which(group == i)
        X.i <- X[group.i,]
        y[group.i] <- X.i %*% beta.i
    }
    y = y + rnorm(l, sd = 5)
    data <- as.data.frame(cbind(group, positions.data, y, X[,-1]))
    colnames(data) <- c("group", "lon", "lat", "y", paste0(rep("g", k.g), 1:k.g), paste0(rep("h", k.h), 1:k.h))
    beta <- as.data.frame(cbind(positions.beta, beta))
    colnames(beta) <- c("lon", "lat", "Intercept", paste0(rep("g", k.g), 1:k.g), paste0(rep("h", k.h), 1:k.h))
    list(
        data = data,
        beta = beta
    )
}

In [9]:
generate.data <- function(n, each, order = c(0,1,2), random = c(0,0,0,0)) {
    k.g <- 1
    k.h <- 2
    s <- ifelse(length(each) == 1, each, length(each))
    l <- ifelse(length(each) == 1, n*n*each, sum(each))
    if (length(each) == 1) {
        group <- rep(1:(n*n), each = each)
    } else {
        group <- rep(1:(n*n), each)
    }
    X <- mvrnorm(l, rep(0, k.g + k.h), diag(1, k.g + k.h, k.g + k.h))
    for (i in 1:(n*n)) {
        group.i <- which(group == i)
        X[group.i,1] <- mean(X[group.i,1])
    }
    X <- cbind(1, X)
    positions.data <- matrix(0, l, 2)
    positions.beta <- matrix(0, n*n, 2)
    for (i in 1:n) {
        for (j in 1:n) {
            p1 = (i - 1)*n + j
            positions.beta[p1,] = c(i,j)
            for (k in 1:each[p1]) {
                p2 = ifelse(p1 > 1, sum(each[1:(p1 - 1)]) + k, k)
                positions.data[p2,] = c(i,j)
            }
        }
    }
    beta.g <- matrix(0, n*n, k.g)
    for (i in 1:n) {
        for (j in 1:n) {
            p = (i - 1)*n + j
            beta = 0
            order.g = order[2]
            for (o1 in 0:order.g) {
                for (o2 in 0:order.g) {
                    beta = beta + runif(1) * i^o1 * j^o2
                }
            }
            beta.g[p, 1] = 10 * beta / (n^(2*order.g))
        }
    }
    beta.g[, 1] = scale(beta.g[, 1] + random[2] * rnorm(n*n, sd = 0.1))
    beta.h <- matrix(0, n*n, k.h)
    for (i in 1:n) {
        for (j in 1:n) {
            p = (i - 1)*n + j
            beta = 0
            order.h = order[3]
            for (o1 in 0:order.h) {
                for (o2 in 0:order.h) {
                    beta = beta + runif(1) * i^o1 * j^o2
                }
            }
            beta.h[p, 1] = beta / (n^(2*order.h))
        }
    }
    beta.h[, 1] = scale(beta.h[, 1] + random[3] * rnorm(n*n, sd = 0.1))
    beta.h[, 2] = 1 + random[4] * rnorm(n*n, sd = 1)
    beta.intercept <- numeric(n*n)
    for (i in 1:n) {
        for (j in 1:n) {
            p = (i - 1)*n + j
            beta = 0
            order.intercept = order[1]
            for (o1 in 0:order.intercept) {
                for (o2 in 0:order.intercept) {
                    beta = beta + runif(1) * i^o1 * j^o2
                }
            }
            beta.intercept[p] = beta / (n^(2*order.intercept))
        }
    }
    beta.intercept = scale(beta.intercept + random[1] * rnorm(n*n, sd = 0.1))
    beta <- cbind(beta.intercept, beta.g, beta.h)
    y <- numeric(l)
    for (i in 1:(n*n)) {
        beta.i <- beta[i,]
        group.i <- which(group == i)
        X.i <- X[group.i,]
        y[group.i] <- X.i %*% beta.i
    }
    y = y + rnorm(l)
    data <- as.data.frame(cbind(group, positions.data, y, X[,-1]))
    colnames(data) <- c("group", "lon", "lat", "y", paste0(rep("g", k.g), 1:k.g), paste0(rep("h", k.h), 1:k.h))
    beta <- as.data.frame(cbind(positions.beta, beta))
    colnames(beta) <- c("lon", "lat", "Intercept", paste0(rep("g", k.g), 1:k.g), paste0(rep("h", k.h), 1:k.h))
    list(
        data = data,
        beta = beta
    )
}

In [31]:
experiment <- function(gen, kernel = "gaussian") {
    gen.data <- gen$data
    coordinates(gen.data) <- ~ lon + lat
    
    # # Regression
    # ## GWR
    model.gwr.formula <- y ~ g1 + h1 + h2
    model.gwr.bw <- bw.gwr(model.gwr.formula, gen.data, kernel = kernel, adaptive = TRUE, parallel.method = "omp")
    model.gwr <- gwr.basic(model.gwr.formula, gen.data, bw = model.gwr.bw, kernel = kernel, adaptive = TRUE, parallel.method = "omp")
    model.gwr.R2 <- model.gwr$GW.diagnostic$gw.R2
    # ## HLM
    model.hlm.formula <- y ~ 1 + g1 + h1 + h2 + (h1 | group)
    model.hlm <- lmer(model.hlm.formula, data = gen.data@data)
    model.hlm.y <- gen.data@data[,all.vars(model.hlm.formula)[1]]
    model.hlm.R2 <- 1 - sqrt(sum(residuals(model.hlm)^2)) / sqrt(sum((model.hlm.y - mean(model.hlm.y))^2))
    model.hlm.coef <- coef(model.hlm)$group
    colnames(model.hlm.coef)[1] <- "Intercept"
    # ## HLM-GWR
    model.hlmgwr.formula.hlm <- y ~ 1 + h1 + h2 + (h1 | group)
    model.hlmgwr.hlm <- lmer(model.hlmgwr.formula.hlm, data = gen.data@data)
    model.hlmgwr.hlm.coef <- coef(model.hlmgwr.hlm)$group
    model.hlmgwr.gwrdata <- cbind(model.hlmgwr.hlm.coef[,"(Intercept)"], aggregate(cbind(lon, lat, g1) ~ group, data = gen$data, mean))
    colnames(model.hlmgwr.gwrdata) <- c("yhlm", "group", "lon", "lat", "g1")
    coordinates(model.hlmgwr.gwrdata) <- ~ lon + lat
    model.hlmgwr.gw.formula <- yhlm ~ g1
    model.hlmgwr.gwr.bw <- bw.gwr(model.hlmgwr.gw.formula, model.hlmgwr.gwrdata, kernel = kernel, adaptive = TRUE)
    model.hlmgwr.gwr <- gwr.basic(model.hlmgwr.gw.formula, model.hlmgwr.gwrdata, bw = model.hlmgwr.gwr.bw, kernel = kernel, adaptive = TRUE)
    model.hlmgwr.gwr.coef <- model.hlmgwr.gwr$SDF@data[,1:(1+length(c("g1")))]
    model.hlmgwr.coef <- cbind(Intercept = model.hlmgwr.gwr.coef[,1], g1 = model.hlmgwr.gwr.coef[,2:(1+length(c("g1")))], model.hlmgwr.hlm.coef[,2:(1+length(c("h1", "h2")))])
    rownames(model.hlmgwr.coef) <- model.hlmgwr.gwrdata$group
    model.hlmgwr.yhat <- numeric(nrow(gen$data))
    for (g in unique(gen$data$group)) {
        group.data <- cbind(1, as.matrix(gen$data[which(gen$data$group == g), names(model.hlmgwr.coef)[-1]]))
        group.beta <- as.matrix(model.hlmgwr.coef[g,])
        group.y <- group.data %*% t(group.beta)
        model.hlmgwr.yhat[which(gen$data$group == g)] <- group.y
    }
    model.hlmgwr.R2 <- 1 - sum((gen$data$y - model.hlmgwr.yhat)^2) / sum((gen$data$y - mean(gen$data$y))^2)
    
    # Compare
    model.gwr.group <- aggregate(cbind(lon, lat, Intercept, g1, h1, h2) ~ group, 
                                 data = cbind(model.gwr$SDF@data, group = gen.data@data$group, model.gwr$SDF@coords), 
                                 mean)
#     data.frame(
#         order.Intercept = as.integer(order[1]),
#         order.g1 = as.integer(order[2]),
#         order.h1 = as.integer(order[3]),
#         random.Intercept = as.integer(random[1]),
#         random.g1 = as.integer(random[2]),
#         random.h1 = as.integer(random[3]),
#         R2.GWR = c(model.gwr.R2),
#         R2.HLM = c(model.hlm.R2),
#         R2.HLMGWR = c(model.hlmgwr.R2),
#         HLMGWR.Intercept = c(sum((model.hlmgwr.coef$Intercept - gen$beta$Intercept)^2)),
#         HLMGWR.g1 = c(sum((model.hlmgwr.coef$g1 - gen$beta$g1)^2)),
#         HLMGWR.h1 = c(sum((model.hlmgwr.coef$h1 - gen$beta$h1)^2)),
#         HLMGWR.h2 = c(sum((model.hlmgwr.coef$h2 - gen$beta$h2)^2)),
#         GWR.Intercept = c(sum((model.gwr.group$Intercept - gen$beta$Intercept)^2)),
#         GWR.g1 = c(sum((model.gwr.group$g1 - gen$beta$g1)^2)),
#         GWR.h1 = c(sum((model.gwr.group$h1 - gen$beta$h1)^2)),
#         GWR.h2 = c(sum((model.gwr.group$h2 - gen$beta$h2)^2)),
#         HLM.Intercept = c(sum((model.hlm.coef$Intercept - gen$beta$Intercept)^2)),
#         HLM.g1 = c(sum((model.hlm.coef$g1 - gen$beta$g1)^2)),
#         HLM.h1 = c(sum((model.hlm.coef$h1 - gen$beta$h1)^2)),
#         HLM.h2 = c(sum((model.hlm.coef$h2 - gen$beta$h2)^2))
#     )
    list(
        GWR = model.gwr.group,
        HLM = model.hlm.coef,
        HLMGWR = model.hlmgwr.coef
    )
}

In [32]:
exp.results = list()
for (oi in 2:3) {
    for (og in 2:2) {
        for (oh in 2:2) {
            for (ri in c(0)) {
                rg <- 0
                rh1 <- 1
                rh2 <- 0
                message(paste(oi, og, oh, ri, rg, rh1, rh2))
                gen <- generate.data(15, floor(runif(15*15, min = 10, max = 50)), order = c(oi, og, oh), random = c(ri, rg, rh1, rh2))
                result = experiment(gen, kernel = "gaussian")
                exp.results[[paste("order", oi, og, oh, sep = ".")]] <- result
            }
        }
    }
}

2 2 2 0 0 1 0



Take a cup of tea and have a break, it will take a few minutes.
          -----A kind suggestion from GWmodel development group
Adaptive bandwidth: 4398 CV score: 20203.51 
Adaptive bandwidth: 2726 CV score: 18935.28 
Adaptive bandwidth: 1692 CV score: 17376.85 
Adaptive bandwidth: 1053 CV score: 15771.58 
Adaptive bandwidth: 658 CV score: 14302.25 
Adaptive bandwidth: 414 CV score: 13239.03 
Adaptive bandwidth: 263 CV score: 12410.75 
Adaptive bandwidth: 170 CV score: 11852.98 
Adaptive bandwidth: 112 CV score: 11292.39 
Adaptive bandwidth: 76 CV score: 11220.34 
Adaptive bandwidth: 54 CV score: 11221.56 
Adaptive bandwidth: 90 CV score: 11221.8 
Adaptive bandwidth: 67 CV score: 11220.34 
Adaptive bandwidth: 146 CV score: 202.2551 
Adaptive bandwidth: 98 CV score: 180.7619 
Adaptive bandwidth: 67 CV score: 156.159 
Adaptive bandwidth: 49 CV score: 134.0416 
Adaptive bandwidth: 37 CV score: 116.1453 
Adaptive bandwidth: 30 CV score: 104.7735 
Adaptive bandwidth: 25 CV score: 94.20018 


3 2 2 0 0 1 0



Take a cup of tea and have a break, it will take a few minutes.
          -----A kind suggestion from GWmodel development group
Adaptive bandwidth: 3967 CV score: 17677.94 
Adaptive bandwidth: 2460 CV score: 16779.57 
Adaptive bandwidth: 1527 CV score: 15661.51 
Adaptive bandwidth: 952 CV score: 14679.19 
Adaptive bandwidth: 595 CV score: 13575.71 
Adaptive bandwidth: 376 CV score: 12855.71 
Adaptive bandwidth: 239 CV score: 12253.82 
Adaptive bandwidth: 156 CV score: 11334.6 
Adaptive bandwidth: 103 CV score: 10707.38 
Adaptive bandwidth: 71 CV score: 10483.84 
Adaptive bandwidth: 51 CV score: 10388.73 
Adaptive bandwidth: 38 CV score: NaN 
Adaptive bandwidth: 58 CV score: 10388.73 
Adaptive bandwidth: 146 CV score: 221.4978 
Adaptive bandwidth: 98 CV score: 204.9272 
Adaptive bandwidth: 67 CV score: 184.2043 
Adaptive bandwidth: 49 CV score: 167.098 
Adaptive bandwidth: 37 CV score: 150.3405 
Adaptive bandwidth: 30 CV score: 139.6236 
Adaptive bandwidth: 25 CV score: 132.5659 
Adapti

In [39]:
beta.estimate <- with(exp.results$`order.3.2.2`, cbind(
    select(GWR, "group":"lat"), 
    GWR = select(GWR, "Intercept":"h2"),
    HLM = HLM,
    HLMGWR = HLMGWR,
    Real = select(gen$beta, "Intercept":"h2")
))
head(beta.estimate)

,group,lon,lat,GWR.Intercept,GWR.g1,GWR.h1,GWR.h2,HLM.Intercept,HLM.g1,HLM.h1,HLM.h2,HLMGWR.Intercept,HLMGWR.g1,HLMGWR.h1,HLMGWR.h2,Real.Intercept,Real.g1,Real.h1,Real.h2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,1,-0.2293197,-0.8466500,-0.5180305,1.0135847,-0.251393136,0.3749695,-0.1644824,0.9930965,-0.3858760,-0.4222580,-0.16391197,0.993107,-0.4343241,-0.6384260,-0.3951571,1
2,2,1,2,-0.2887019,-1.0935313,-0.6854783,1.0726031,-0.933139266,0.3749695,-0.7993493,0.9930965,-0.3949376,-0.4375435,-0.79378529,0.993107,-0.4343042,-0.6368699,-0.5595338,1
3,3,1,3,-0.4310230,-0.5367368,-0.7530058,1.0868687,-0.417303809,0.3749695,-0.0488617,0.9930965,-0.4020094,-0.4592127,-0.04779983,0.993107,-0.4342838,-0.6350389,0.2877351,1
4,4,1,4,-0.4755865,-0.2610251,-0.6767015,1.0616405,-0.434404388,0.3749695,-0.4328190,0.9930965,-0.3987533,-0.4903713,-0.43839899,0.993107,-0.4341890,-0.6312450,-0.3807783,1
5,5,1,5,-0.3960891,-0.4176080,-0.6066725,1.0074757,-0.788777938,0.3749695,-0.7680008,0.9930965,-0.3949426,-0.5228562,-0.76640881,0.993107,-0.4340578,-0.6299493,-0.7838180,1
6,6,1,6,-0.3733415,-0.5367494,-0.4695494,0.9453379,-0.001638551,0.3749695,-0.7739104,0.9930965,-0.3907735,-0.5535001,-0.77651262,0.993107,-0.4340390,-0.6322030,-1.0783974,1


In [65]:
save(beta.estimate, file = "Data/beta.estimate.rds")

In [62]:
summary(lm(Real.g1 ~ HLMGWR.g1, beta.estimate))$adj.r.squared
summary(lm(Real.g1 ~ GWR.g1, beta.estimate))$adj.r.squared

[1] 0.5492409

[1] 0.0938235

In [66]:
gen$data

group,lon,lat,y,g1,h1,h2
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,0.354244555,0.1270536,0.062213439,0.46742472
1,1,1,-2.408138621,0.1270536,0.474557110,-1.07626799
1,1,1,-1.111993687,0.1270536,1.598007700,-0.25128305
1,1,1,1.939195401,0.1270536,-2.029720624,1.58606397
1,1,1,1.474982332,0.1270536,0.525826607,0.78922588
1,1,1,0.583830326,0.1270536,0.201293115,0.93734181
1,1,1,-1.708575312,0.1270536,1.768363153,-0.85921633
1,1,1,1.671140040,0.1270536,-0.758292627,1.17408544
1,1,1,-1.160187349,0.1270536,-0.193676746,-1.47758302


filter(exp.results, random.Intercept == 0 & random.g1 == 0 & random.h1 == 0) %>% with(rbind(
    data.frame(cbind(order.Intercept, order.g1, order.h1), R2 = R2.GWR, Label = "GWR"),
    data.frame(cbind(order.Intercept, order.g1, order.h1), R2 = R2.HLM, Label = "HLM"),
    data.frame(cbind(order.Intercept, order.g1, order.h1), R2 = R2.HLMGWR, Label = "HLMGWR")
))　%>% ggplot(aes(x = order.Intercept, y = R2, fill = Label)) +
    geom_bar(position = "dodge", stat = "identity") +
    facet_wrap(vars(order.g1, order.h1))

filter(exp.results, random.Intercept == 0 & random.g1 == 0 & random.h1 == 0) %>% with(rbind(
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "GWR", Coef = "Intercept", MSE = GWR.Intercept),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "GWR", Coef = "g1", MSE = GWR.g1),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "GWR", Coef = "h1", MSE = GWR.h1),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "GWR", Coef = "h2", MSE = GWR.h2),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "HLM", Coef = "Intercept", MSE = HLM.Intercept),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "HLM", Coef = "g1", MSE = HLM.g1),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "HLM", Coef = "h1", MSE = HLM.h1),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "HLM", Coef = "h2", MSE = HLM.h2),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "HLMGWR", Coef = "Intercept", MSE = HLMGWR.Intercept),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "HLMGWR", Coef = "g1", MSE = HLMGWR.g1),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "HLMGWR", Coef = "h1", MSE = HLMGWR.h1),
    data.frame(cbind(order.Intercept, order.g1, order.h1), Label = "HLMGWR", Coef = "h2", MSE = HLMGWR.h2)
)) %>% ggplot(aes(x = Coef, y = MSE, fill = Label)) +
    geom_bar(position = "dodge", stat = "identity") +
    facet_wrap( ~ order.h1 + order.g1 + order.Intercept) +
    scale_y_log10()